In [1]:
import numpy as np
import pandas as pd

#### Create Company Price TimeSeries

In [2]:
df = pd.read_csv('S&P500_component_data/S&P500_company_components_prices.csv')

commissions = pd.read_csv('full_timeseries_orders.csv')

IOError: File S&P500_component_data/S&P500_company_components_prices.csv does not exist

In [ ]:
names_mapping = pd.read_csv('s&p500_conm_and_generic.csv')[['conm','generic']]
names_mapping

In [ ]:
COMPANY_NAME = names_mapping['conm'].loc[0]
GENERIC_NAME = names_mapping['generic'].loc[0]
#GENERIC_NAME = 'amazon'

df1 = df[df['conm']==COMPANY_NAME]
df1['date']= pd.to_datetime(df1['datadate'],format='%Y%m%d')
df1.head()

#### Align with commissions

#### Generalize

In [ ]:
def timeseries_regready(commissions,df1):
    commissions_nord = commissions[commissions['generic_name']==GENERIC_NAME]

    commissions_nord.drop('generic_name',axis=1,inplace=True)
    commissions_nord=commissions_nord.transpose()
    print(commissions_nord)
    commissions_nord.reset_index(inplace=True)
    commissions_nord=commissions_nord.rename(columns={9:'order_tot','index':'date'}) #The column number becomes the column name upon transpose (currently changed manually at each aggregation)
    commissions_nord['date'] = pd.to_datetime(commissions_nord['date'],format='%d/%m/%y')
    
    #for etsy (otherwise remove)
    #commissions_nord = commissions_nord[commissions_nord['date']>pd.datetime(2015,05,01)]
    #commissions_nord = commissions_nord.reset_index(drop=True)
    
    comms_dates = commissions_nord['date'] + pd.DateOffset(1)
    
    cols = df1.columns
    weekly_prices = pd.DataFrame(columns=cols)
    
    for date in comms_dates:
        a = df1[df1['date']==date]
        while a.empty:
            a = df1[df1['date']==date+pd.DateOffset(days=1)]
        a['sunday_date'] = date - pd.DateOffset(days=1)
        weekly_prices=weekly_prices.append(a)
        
    weekly_prices.reset_index(drop=True, inplace=True)
    
    weekly_prices = weekly_prices.rename(columns={'prccd':'Close','prcod':'Open'})
    
    weekly_prices1 = weekly_prices

    weekly_prices1['open_next_wk'] = weekly_prices1['Open'].shift(-1)
    weekly_prices1['weekly_ret'] = weekly_prices1['open_next_wk'] - weekly_prices1['Open']
    weekly_prices1['weekly_pct'] = weekly_prices1['weekly_ret']/weekly_prices1['Open']
    weekly_prices1['sunday_date_beginning'] = weekly_prices1['sunday_date']
    
    weekly_prices1 = weekly_prices1[['conm','sunday_date_beginning','Open','weekly_ret','weekly_pct']]
    
    new_df = []

    for idx in range(6,177): #177 if not ETSY
        most_recent_comm = []
        for i in range(6):
            #print commissions_nord['order_tot']
            most_recent_comm.insert(0,commissions_nord['order_tot'][idx-i])
        most_recent_comm.append(weekly_prices1['sunday_date_beginning'][idx])
        new_df.append(most_recent_comm)
        
    new_df = pd.DataFrame(new_df,columns=['six','five','four','three','two','one','sunday_beginning'])
    
    merged = new_df.merge(weekly_prices1,how='left',left_on='sunday_beginning',right_on='sunday_date_beginning')
    
    merged['monthly_ret']=0.0
    for idx2 in range(168): #168 if not ETSY
        merged['monthly_ret'][idx2] = merged['weekly_ret'][idx2]+merged['weekly_ret'][idx2+1]+merged['weekly_ret'][idx2+2]+merged['weekly_ret'][idx2+3]
    merged['monthly_pct_ret'] = merged['monthly_ret']/merged['Open']
        
    merged['is_DEC'] = merged['sunday_beginning'].dt.month==12
    merged['is_NOV'] = merged['sunday_beginning'].dt.month==11
        
    return merged
    
    
    
    

In [ ]:
merged = timeseries_regready(commissions,df1)
merged = merged[:-5]
merged['month'] = 0
merged['month'] = merged['sunday_beginning'].map(lambda x:x.month)
merged

In [ ]:
merged_average_month = merged.groupby(by='month',as_index=False).mean()
merged_average_month = merged_average_month[['month','one','two','three','four','five','six']]
merged_average_month

In [ ]:
merged_average_month.rename(columns={'one':'one_avg','two':'two_avg','three':'three_avg','four':'four_avg',
                                     'five':'five_avg','six':'six_avg'},inplace=True)

merged_average_month

std_df2 = pd.merge(merged,merged_average_month,how='left',left_on='month',right_on='month')

std_df2['one'] = std_df2['one']/std_df2['one_avg']
std_df2['two'] = std_df2['two']/std_df2['two_avg']
std_df2['three'] = std_df2['three']/std_df2['three_avg']
std_df2['four'] = std_df2['four']/std_df2['four_avg']
std_df2['five'] = std_df2['five']/std_df2['five_avg']
std_df2['six'] = std_df2['six']/std_df2['six_avg']

std_df2



In [ ]:
#std_df = merged.copy()
#std_df2 = merged.copy()
#
#lag=52
#
#for idx,row in enumerate(std_df['sunday_beginning']):
#    if idx>=lag:
#        std_df2['one'][idx] = std_df['one'][idx]/std_df['one'][idx-lag]
#        std_df2['two'][idx] = std_df['two'][idx]/std_df['two'][idx-lag]
#        std_df2['three'][idx] = std_df['three'][idx]/std_df['three'][idx-lag]
#        std_df2['four'][idx] = std_df['four'][idx]/std_df['four'][idx-lag]
#        std_df2['five'][idx] = std_df['five'][idx]/std_df['five'][idx-lag]
#        std_df2['six'][idx] = std_df['six'][idx]/std_df['six'][idx-lag]

In [ ]:
comp_short = COMPANY_NAME.split()[0]
comp_short = comp_short.lower()
std_df2.to_csv('reg_ready_'+comp_short+'.csv',index=False)